In [1]:
import asyncio
from decimal import ROUND_DOWN, Decimal

from starknet_py.net.full_node_client import FullNodeClient

def health_ratio_is_valid(health_ratio_level: Decimal) -> bool:
    """
    Checks if the given health ratio level is valid.
    :param health_ratio_level: Health ratio level
    :return: bool
    """
    return health_ratio_level > Decimal("0") and health_ratio_level != Decimal(
        "Infinity"
    )

def format_health_ratio(ratio: Decimal) -> str:
    """
    Format health_ratio_level to 2 symbols after dot.
    :param ratio: Health ratio level
    :return: str
    """
    if ratio == Decimal("Infinity"):
        return "Infinity"
    return f"{ratio.quantize(Decimal('0.01'), rounding=ROUND_DOWN)}"

def calculate_health_ratio(
    borrowing_results: list[tuple], deposit_results: list[tuple]
) -> dict:
    """
    Calculate health_ratio_level
    :param borrowing_results: borrowing_data, deposit_results: deposit_data
    :return: dict
    """
    total_borrowed = {}
    total_deposited = {}

    # Summary raw_amount from Borrowing Results
    for user, raw_amount, _ in borrowing_results:
        if user not in total_borrowed:
            total_borrowed[user] = Decimal(0)
        total_borrowed[user] += raw_amount

    # Summary of face_amount from deposit results
    for user, face_amount in deposit_results:
        if user not in total_deposited:
            total_deposited[user] = Decimal(0)
        total_deposited[user] += face_amount

    # Calculate health_ratio for each user
    health_ratios = {}
    for user in total_borrowed:
        borrowed = total_borrowed[user]
        deposited = total_deposited.get(user, Decimal(0))
        if borrowed > 0:
            health_ratios[user] = deposited / borrowed
        else:
            health_ratios[user] = Decimal("Infinity")  # if not borrows

    for user, ratio in health_ratios.items():
        if health_ratio_is_valid(ratio):
            formatted_ratio = format_health_ratio(ratio)
            print(f"Health ratio for user {user} is valid: {formatted_ratio}")
        else:
            print(f"Health ratio for user {user} is invalid: {ratio}")

    return health_ratios

async def main():
    node = FullNodeClient(node_url="http://178.32.172.148:6060")

    borrowing_key_output = [
        442150749710818778059430935486738358608566890420557042714133150282782342977
    ]
    deposit_key_output = [
        256702315380875556620535413554505018969076471135601079992472732711650693874
    ]

    borrowing_selector = (
        "0xfa3f9acdb7b24dcf6d40d77ff2f87a87bca64a830a2169aebc9173db23ff41"
    )
    deposit_selector = (
        "0x9149d2123147c5f43d258257fef0b7b969db78269369ebcf5ebb9eef8592f2"
    )

    # Query
    deposit_and_borrowing_events = await node.get_events(
        address="0x04c0a5193d58f74fbace4b74dcf65481e734ed1714121bdc571da345540efa05",
        from_block_number=850152,
        to_block_number=860000,
        keys=[[deposit_selector, borrowing_selector]],
        chunk_size=9999,
    )

    # Initialize lists to store results
    borrowing_results = []
    deposit_results = []

    # Extracting the events
    for event in deposit_and_borrowing_events.events:
        # Check for borrowing event using the selector
        if borrowing_key_output[0] in event.keys:
            user = event.data[0]  # Assuming data[0] is user
            raw_amount = event.data[2]  # Assuming data[2] is raw_amount
            face_amount = event.data[3]  # Assuming data[3] is face_amount
            borrowing_results.append(
                (
                    user,
                    raw_amount,
                    face_amount,
                )
            )

        # Check for deposit event using the selector
        elif deposit_key_output[0] in event.keys:
            user = event.data[0]  # Assuming data[0] is user
            face_amount = event.data[2]  # Assuming data[2] is face_amount
            deposit_results.append((user, face_amount))

    health_ratio_results = calculate_health_ratio(borrowing_results, deposit_results)

    return health_ratio_results

# Running asyncio in a notebook
def run_main():
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # For Jupyter, we need to create a new task if the loop is already running
        return asyncio.ensure_future(main())
    else:
        return loop.run_until_complete(main())

# Run the main function
health_ratios = run_main()


SyntaxError: invalid syntax (394388680.py, line 1)